In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

import statsmodels.api as sm

In [3]:
# Load Dataset

df = pd.read_csv('./datasets/happiness_2019.csv')

In [4]:
# Get descriptive statistics for the dataset

df.describe(include='all')

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
count,156.000000,156,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000,156.000000
unique,NaN,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,78.500000,NaN,5.407096,0.905147,1.208814,0.725244,0.392571,0.184846,0.110603
std,45.177428,NaN,1.113120,0.398389,0.299191,0.242124,0.143289,0.095254,0.094538
min,1.000000,NaN,2.853000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,39.750000,NaN,4.544500,0.602750,1.055750,0.547750,0.308000,0.108750,0.047000
50%,78.500000,NaN,5.379500,0.960000,1.271500,0.789000,0.417000,0.177500,0.085500
75%,117.250000,NaN,6.184500,1.232500,1.452500,0.881750,0.507250,0.248250,0.141250


In [5]:
# Check for duplicates

df.duplicated().sum()

0

In [6]:
# Check for missing values

df.isna().sum()

Overall rank                    0
Country or region               0
Score                           0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
dtype: int64

In [7]:
# Remove rows with missing values and check again

clean_df = df.dropna()

clean_df.isna().sum()

Overall rank                    0
Country or region               0
Score                           0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
dtype: int64

In [9]:
# Rename columns to be a single word
clean_df.columns = clean_df.columns.str.lower().str.replace(' ','_')

clean_df.columns

Index(['overall_rank', 'country_or_region', 'score', 'gdp_per_capita',
       'social_support', 'healthy_life_expectancy',
       'freedom_to_make_life_choices', 'generosity',
       'perceptions_of_corruption'],
      dtype='object')

In [14]:
# Create axis and sets

X = clean_df[['gdp_per_capita', 'social_support', 'healthy_life_expectancy', 'perceptions_of_corruption']]
y = clean_df['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random.randint(0, 100_000))

In [15]:
# Train the model

model = LinearRegression()
model.fit(X_train, y_train)

r_sq = model.score(X_train, y_train)

In [16]:
# Predict the model and visualize the results

y_pred_train = model.predict(X_test)

r_sq = r2_score(y_test, y_pred_train)
mae = mean_absolute_error(y_test, y_pred_train)
mse = mean_squared_error(y_test, y_pred_train)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_train))

x = sm.add_constant(X)
results = sm.OLS(y,x).fit()
results.summary()

In [17]:
# Check the coefficients

cdf = pd.DataFrame(model.coef_, X.columns, columns=['Coefficients'])
cdf

,Coefficients
gdp_per_capita,0.649560
social_support,1.431894
healthy_life_expectancy,1.290839
perceptions_of_corruption,1.767437


In [ ]:
# Generate results and save them to a JSON file

result = {
    'data': clean_df.to_dict(),
    'context': 'The dataset contains the happiness score of 156 countries based on the factors of GDP per capita, social support, healthy life expectancy, and perceptions of corruption. The goal is to predict the happiness score based on these factors.',
    'metrics': {
        'Coefficients': cdf.to_dict()['Coefficients'],
        'R2': r_sq,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        "Reference": "score"
    }
}

with open('lr_happiness_result.json', 'w') as file:
    file.write(json.dumps(result, indent=4))